In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import client_surveillance_jupyter

In [4]:
COMPANY_IDENTIFIER = ['CI']
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [5]:
#fetch download report and lisences
company_licenses_query = create_queries.create_company_licenses_query(COMPANY_IDENTIFIER)
company_download_summaries_query = create_queries.create_company_download_summaries_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)

company_licenses_dataframe = pd.read_sql_query(company_licenses_query, engine)
company_download_summaries_dataframe = pd.read_sql_query(company_download_summaries_query, engine)

## Check download status summaries

In [6]:
license_numbers = company_download_summaries_dataframe['license_number'].unique()
download_summary_records = company_download_summaries_dataframe.to_dict('records')

In [7]:
license_numbers

array(['C11-0000032-LIC'], dtype=object)

# For Claybourne

In [8]:
distro_license_numbers = ['C11-0000032-LIC']
all_license_numbers = ['C11-0000032-LIC']

## Pull data

## distro inventory

In [9]:
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=distro_license_numbers,
)
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)
df_inventory = company_inventory_packages_dataframe
df_inventory['license_number'].unique()

array(['C11-0000032-LIC'], dtype=object)

In [121]:
print(company_inventory_packages_query)


		select
			metrc_packages.license_number,
			metrc_packages.package_id,
			metrc_packages.package_label,
			metrc_packages.type,
			metrc_packages.packaged_date,
			metrc_packages.last_modified_at,
			metrc_packages.package_type,
			metrc_packages.product_name,
			metrc_packages.product_category_name,
			metrc_packages.quantity,
			metrc_packages.unit_of_measure,
			metrc_packages.package_payload.itemid as item_id,
			metrc_packages.package_payload.itemproductcategorytype as item_product_category_type,
			metrc_packages.package_payload.productionbatchnumber as production_batch_number,
			metrc_packages.package_payload.sourceproductionbatchnumbers as source_production_batch_numbers,
			metrc_packages.package_payload.sourceharvestnames as source_harvest_names,
			metrc_packages.package_payload.istestingsample as is_testing_sample,
			metrc_packages.package_payload.istradesample as is_trade_sample,
			metrc_packages.package_payload.isonhold as is_on_hold,
			metrc_packages.package_paylo

## distro incoming = incoming

In [10]:
all_incoming_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=all_license_numbers,
)

df_in_all = pd.read_sql_query(all_incoming_query, engine)
df_in_all['license_number'].unique()

array(['C11-0000032-LIC'], dtype=object)

## distro outgoing = outgoing

In [11]:
distro_outgoing_query = create_queries.create_company_outgoing_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=distro_license_numbers,
)

df_out_distro = pd.read_sql_query(distro_outgoing_query, engine)
df_out_distro['license_number'].unique()

array(['C11-0000032-LIC'], dtype=object)

# look at distro incoming data

In [ ]:
df_in_distro['created_date'] = pd.to_datetime(df_in_distro['created_date'])
df_in_distro['create_year'] = [df_in_distro['created_date'][i].year for i in range(len(df_in_distro))]


In [ ]:
# how many from internal vs external?
df_in_distro[['create_year','delivery_type','package_id']].groupby(['create_year','delivery_type']).count()

In [ ]:
# who are they buying from?
df_in_distro[df_in_distro['delivery_type'] == 'INCOMING_FROM_VENDOR']['shipper_facility_license_number'].unique()

In [ ]:
# top parties they buy from in 2022
df_in_distro[df_in_distro['create_year'] == 2022][['shipper_facility_name','shipper_facility_license_number','package_id']].\
groupby(['shipper_facility_name','shipper_facility_license_number']).count().sort_values(by = 'package_id',ascending = False)

In [ ]:
# df_in_distro_2022 = df_in_distro[(df_in_distro['created_date'] >= '2022-06-01')&\
#                                  (~df_in_distro['received_datetime'].isna()&
#                                   (df_in_distro['receiver_wholesale_price'].isna()))]

In [ ]:
#df_in_distro_2022['product_name'].to_dict().values()

# look at distro outgoing data

In [12]:
df_out_distro['created_date'] = pd.to_datetime(df_out_distro['created_date'])
df_out_distro['create_year'] = [df_out_distro['created_date'][i].year for i in range(len(df_out_distro))]


In [80]:
df_out_distro.head()

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name,create_year,per_unit_incoming
0,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120490,2022-10-01,2022-10-02 15:07:47+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000072-LIC,2 RIVER LABS,Transfer,Standard,29369135,1A406030000326B000146429,transfer,1A406030000326B000146192,13 9/16/2022,Accepted,True,False,Flower,Claybourne Bulk Flower - Eagle MAC,failed,NaN,20.0,Grams,20.0,Grams,NaN,NaN,None,2022,NaN
1,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004121039,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0001041-LIC,TAT FRESNO II LLC,Wholesale Manifest,Wholesale,29371682,1A406030000326B000146452,transfer,1A406030000326B000143460,"DB - S012522 - Room 1 Harvest, G - S020122 - R...",Shipped,False,False,Flower (packaged - each),Claybourne Co. Power Pack 4.5g - Sample - Doub...,passed,2.00,2.0,Each,NaN,None,NaN,4.5,Grams,2022,1.00
2,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004121039,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0001041-LIC,TAT FRESNO II LLC,Wholesale Manifest,Wholesale,29371684,1A406030000326B000146458,transfer,1A406030000326B000146284,"003-7/11/2022, DB - S062122 - Room 4 Harvest",Shipped,False,False,Pre-Roll Flower,Claybourne Co. Flyers Full Flower .5g 2 Pack -...,passed,180.00,30.0,Each,NaN,None,NaN,1.0,Grams,2022,6.00
3,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120492,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C9-0000384-LIC,OPTIMAL BUSINESS VENTURES LLC,Wholesale Manifest,Wholesale,29369813,1A406030000326B000146432,transfer,1A406030000326B000144146,21 8/1/2022,Shipped,False,False,Flower (packaged eighth - each),Claybourne Co. 3.5g - Sample - Chemistry,passed,0.01,1.0,Each,NaN,None,NaN,3.5,Grams,2022,0.01
4,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120807,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C9-0000384-LIC,OPTIMAL BUSINESS VENTURES LLC,Wholesale Manifest,Wholesale,29370650,1A406030000326B000146437,transfer,1A406030000326B000144145,21 8/1/2022,Shipped,False,False,Flower (packaged eighth - each),Claybourne Co. 3.5g - (Case of 16) - Chemistry,passed,360.00,16.0,Each,NaN,None,NaN,3.5,Grams,2022,22.50


## pricing coverage in outgoing transfers data

In [18]:
# pricing coverage
df_out_distro[~df_out_distro['receiver_wholesale_price'].isna()].shape[0] / df_out_distro.shape[0]

0.9430997430884325

In [92]:
df_out_distro



,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name,create_year,per_unit_incoming
0,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120490,2022-10-01,2022-10-02 15:07:47+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000072-LIC,2 RIVER LABS,Transfer,Standard,29369135,1A406030000326B000146429,transfer,1A406030000326B000146192,13 9/16/2022,Accepted,True,False,Flower,Claybourne Bulk Flower - Eagle MAC,failed,NaN,20.0,Grams,20.0,Grams,NaN,NaN,None,2022,NaN
1,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004121039,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0001041-LIC,TAT FRESNO II LLC,Wholesale Manifest,Wholesale,29371682,1A406030000326B000146452,transfer,1A406030000326B000143460,"DB - S012522 - Room 1 Harvest, G - S020122 - R...",Shipped,False,False,Flower (packaged - each),Claybourne Co. Power Pack 4.5g - Sample - Doub...,passed,2.00,2.0,Each,NaN,None,NaN,4.5,Grams,2022,1.000000
2,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004121039,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0001041-LIC,TAT FRESNO II LLC,Wholesale Manifest,Wholesale,29371684,1A406030000326B000146458,transfer,1A406030000326B000146284,"003-7/11/2022, DB - S062122 - Room 4 Harvest",Shipped,False,False,Pre-Roll Flower,Claybourne Co. Flyers Full Flower .5g 2 Pack -...,passed,180.00,30.0,Each,NaN,None,NaN,1.0,Grams,2022,6.000000
3,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120492,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C9-0000384-LIC,OPTIMAL BUSINESS VENTURES LLC,Wholesale Manifest,Wholesale,29369813,1A406030000326B000146432,transfer,1A406030000326B000144146,21 8/1/2022,Shipped,False,False,Flower (packaged eighth - each),Claybourne Co. 3.5g - Sample - Chemistry,passed,0.01,1.0,Each,NaN,None,NaN,3.5,Grams,2022,0.010000
4,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120807,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C9-0000384-LIC,OPTIMAL BUSINESS VENTURES LLC,Wholesale Manifest,Wholesale,29370650,1A406030000326B000146437,transfer,1A406030000326B000144145,21 8/1/2022,Shipped,False,False,Flower (packaged eighth - each),Claybourne Co. 3.5g - (Case of 16) - Chemistry,passed,360.00,16.0,Each,NaN,None,NaN,3.5,Grams,2022,22.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123773,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000227096,2020-01-02,2020-01-04 00:31:56+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000121-LIC,"ECSD MANAGEMENT, LLC.",Wholesale Manifest,Wholesale,1794252,1A406030000326B000004277,transfer,1A406030000326B000000555,F1.10 6/11/19 LAVA_Lava Cake_2019-09-24,Accepted,False,False,Flower,Claybourne Packaged Flower - Lava Cake - 3.5g ...,passed,672.00,112.0,Grams,112.0,Grams,672.00,NaN,None,2020,6.000000
123774,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000227096,2020-01-02,2020-01-04 00:31:56+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000121-LIC,"ECSD MANAGEMENT, LLC.",Wholesale Manifest,Wholesale,1794271,1A406030000326B000004284,transfer,1A406030000326B000002909,P02QST91007,Accepted,False,False,Flower,Claybourne Packaged Flower - 3x Sour - 1g - (C...,passed,350.00,50.0,Grams,50.0,Grams,350.00,NaN,None,2020,7.000000
123775,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000227025,2020-01-02,2020-01-04 00:32:33+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000345-LIC,"THE AIRPORT COLLECTIVE, INC.",Wholesale Manifest,Wholesale,1792505,1A406030000326B000004268,transfer,1A406030000326B000002402,F6.10 7/2/19 KC7_Kush Cake #7_2019-11-04,Accepted,False

In [94]:
df_out_distro_prod_cat_coverage = df_out_distro[['product_category_name','shipped_unit_of_measure','shipper_wholesale_price']].groupby(['product_category_name','shipped_unit_of_measure']).\
apply(lambda x: x[x['shipper_wholesale_price'].notnull()].shape[0] / x.shape[0]).to_frame().reset_index()
df_out_distro_prod_cat_coverage.columns = ['product_category_name','shipped_unit_of_measure','coverage']
df_out_distro_prod_cat_coverage.sort_values(by = 'coverage',ascending = False)


,product_category_name,shipped_unit_of_measure,coverage
11,Flower (packaged ounce - each),Each,1.000000
10,Flower (packaged half ounce - each),Each,0.999844
20,Shake (Packaged Quarter - each),Each,0.999101
9,Flower (packaged gram - each),Each,0.998198
8,Flower (packaged eighth - each),Each,0.997449
12,Flower (packaged quarter - each),Each,0.996556
7,Flower (packaged - each),Each,0.994247
4,Extract (weight - each),Each,0.979460
16,Other Concentrate (weight - each),Each,0.978020
18,Pre-Roll Infused,Each,0.971790


In [113]:
s = set(df_out_distro[(df_out_distro['product_category_name'] == 'Flower')&(~df_out_distro['shipper_wholesale_price'].isna())]['product_name'].unique())

In [81]:
df_out_distro

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name,create_year,per_unit_incoming
0,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120490,2022-10-01,2022-10-02 15:07:47+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000072-LIC,2 RIVER LABS,Transfer,Standard,29369135,1A406030000326B000146429,transfer,1A406030000326B000146192,13 9/16/2022,Accepted,True,False,Flower,Claybourne Bulk Flower - Eagle MAC,failed,NaN,20.0,Grams,20.0,Grams,NaN,NaN,None,2022,NaN
1,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004121039,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0001041-LIC,TAT FRESNO II LLC,Wholesale Manifest,Wholesale,29371682,1A406030000326B000146452,transfer,1A406030000326B000143460,"DB - S012522 - Room 1 Harvest, G - S020122 - R...",Shipped,False,False,Flower (packaged - each),Claybourne Co. Power Pack 4.5g - Sample - Doub...,passed,2.00,2.0,Each,NaN,None,NaN,4.5,Grams,2022,1.000000
2,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004121039,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0001041-LIC,TAT FRESNO II LLC,Wholesale Manifest,Wholesale,29371684,1A406030000326B000146458,transfer,1A406030000326B000146284,"003-7/11/2022, DB - S062122 - Room 4 Harvest",Shipped,False,False,Pre-Roll Flower,Claybourne Co. Flyers Full Flower .5g 2 Pack -...,passed,180.00,30.0,Each,NaN,None,NaN,1.0,Grams,2022,6.000000
3,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120492,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C9-0000384-LIC,OPTIMAL BUSINESS VENTURES LLC,Wholesale Manifest,Wholesale,29369813,1A406030000326B000146432,transfer,1A406030000326B000144146,21 8/1/2022,Shipped,False,False,Flower (packaged eighth - each),Claybourne Co. 3.5g - Sample - Chemistry,passed,0.01,1.0,Each,NaN,None,NaN,3.5,Grams,2022,0.010000
4,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120807,2022-10-01,NaT,C11-0000032-LIC,CI DISTRIBUTION LLC,C9-0000384-LIC,OPTIMAL BUSINESS VENTURES LLC,Wholesale Manifest,Wholesale,29370650,1A406030000326B000146437,transfer,1A406030000326B000144145,21 8/1/2022,Shipped,False,False,Flower (packaged eighth - each),Claybourne Co. 3.5g - (Case of 16) - Chemistry,passed,360.00,16.0,Each,NaN,None,NaN,3.5,Grams,2022,22.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123773,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000227096,2020-01-02,2020-01-04 00:31:56+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000121-LIC,"ECSD MANAGEMENT, LLC.",Wholesale Manifest,Wholesale,1794252,1A406030000326B000004277,transfer,1A406030000326B000000555,F1.10 6/11/19 LAVA_Lava Cake_2019-09-24,Accepted,False,False,Flower,Claybourne Packaged Flower - Lava Cake - 3.5g ...,passed,672.00,112.0,Grams,112.0,Grams,672.00,NaN,None,2020,6.000000
123774,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000227096,2020-01-02,2020-01-04 00:31:56+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000121-LIC,"ECSD MANAGEMENT, LLC.",Wholesale Manifest,Wholesale,1794271,1A406030000326B000004284,transfer,1A406030000326B000002909,P02QST91007,Accepted,False,False,Flower,Claybourne Packaged Flower - 3x Sour - 1g - (C...,passed,350.00,50.0,Grams,50.0,Grams,350.00,NaN,None,2020,7.000000
123775,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000227025,2020-01-02,2020-01-04 00:32:33+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000345-LIC,"THE AIRPORT COLLECTIVE, INC.",Wholesale Manifest,Wholesale,1792505,1A406030000326B000004268,transfer,1A406030000326B000002402,F6.10 7/2/19 KC7_Kush Cake #7_2019-11-04,Accepted,False

In [85]:
# what product cats are going out? -- packaged flowers
df_out_distro[['shipped_unit_of_measure','product_category_name','shipped_quantity']].groupby(\
            ['product_category_name','shipped_unit_of_measure']).sum().sort_values(by = 'shipped_quantity',ascending=False).reset_index()

,product_category_name,shipped_unit_of_measure,shipped_quantity
0,Flower,Grams,4442364.191
1,Leaf,Grams,1021197.150
2,Flower (packaged eighth - each),Each,701842.000
3,Pre-Roll Flower,Each,573395.000
4,Flower (packaged gram - each),Each,249669.000
5,Flower (packaged - each),Each,178965.000
6,Other Concentrate (weight - each),Each,152121.000
7,Kief,Grams,110572.000
8,Pre-Roll Infused,Each,93115.000
9,Flower (packaged half ounce - each),Each,92072.000


In [83]:
# what product cats are going out? -- packaged 
df_out_distro[['manifest_number','product_category_name']].groupby(\
            ['product_category_name']).count().sort_values(by = 'manifest_number',ascending=False)

,manifest_number
product_category_name,
Flower (packaged eighth - each),38804
Flower,31599
Flower (packaged - each),9561
Pre-Roll Flower,8338
Flower (packaged gram - each),6658
Other Concentrate (weight - each),6506
Flower (packaged half ounce - each),6428
Extract (weight - each),4333
Flower (packaged ounce - each),3812


# Inventory

In [24]:
today = date.today()
today

datetime.date(2022, 10, 3)

In [25]:
COMPANY_IDENTIFIER

['CI']

In [26]:
distro_license_numbers

['C11-0000032-LIC']

In [31]:
# what's in their inventory? -- Flower?
df_inventory[['package_id','product_category_name']].groupby('product_category_name').count().\
sort_values(by = 'package_id',ascending = False)

,package_id
product_category_name,
Flower,425
Leaf,297
Flower (packaged eighth - each),127
Pre-Roll Flower,60
Other Concentrate (weight - each),55
Pre-Roll Infused,45
Extract (weight - each),44
Flower (packaged half ounce - each),25
Flower (packaged - each),21


## 1. distro inventory + sales based (distro to retail)

In [95]:
inv_val_distro_to_retail,inv_all_distro_to_retail = client_surveillance_jupyter.calculate_inventory_valuation(df_out_distro,df_inventory,license_numbers,today)

#make sense for CPG


In [29]:
inv_val_distro_to_retail


,date,value,value_after_tax,total_incoming,total,coverage,license,legal_name
0,2022-10-03,1012245.25,1277959.63,325,1172,0.28,[C11-0000032-LIC],2 RIVER LABS


# what product category are we missing coverage the most?

In [46]:
for n in df_inventory['product_category_name'].unique():
    print(n)
    test_cat_name = n
    if df_out_distro[df_out_distro['product_category_name'] == n].shape[0] == 0:
        continue
    test_val,test_ = client_surveillance_jupyter.calculate_inventory_valuation(\
    df_out_distro[df_out_distro['product_category_name'] == test_cat_name],df_inventory[df_inventory['product_category_name'] == test_cat_name],\
    license_numbers,today)
    print(test_val['coverage'].values[0])

Flower (packaged eighth - each)
0.98
Flower (packaged - each)
0.95
Flower
0.0
Pre-Roll Flower
0.58
Flower (packaged gram - each)
0.8
Other Concentrate (weight - each)
0.16
Flower (packaged quarter - each)
0.89
Leaf
0.0
Pre-Roll Infused
0.58
Extract (weight - each)
0.8
Flower (packaged ounce - each)
1.0
Flower (packaged half ounce - each)
0.96
Shake
0.0
Kief
0.0
Shake (Packaged Quarter - each)
1.0
Extract (weight)


In [98]:
# missing a lot on the largest cat in inventory - flower
inv_name_set = set(df_inventory[df_inventory['product_category_name'] == 'Flower']['product_name'].unique())

In [119]:
for i in s:
    print(i)

Claybourne Packaged Flower - Orange Creamsicle - 14g - (Case of 8)
Claybourne Packaged Flower - Oakland Purple - 28g - (Case of 4)
Claybourne Grinds - OG Kush - Hybrid - 7g - Sample
Claybourne Packaged Flower - Sundae Driver - 28g - (Case of 4)
Claybourne Packaged Flower - Frosted Cherry Cookies - 28g - (Case of 4)
Claybourne Packaged Flower - Strawberry Bang - 3.5g - Sample
Claybourne Power Pack - WiFi x Hybrid Kief - 3.5g - Flower - (Case of 20)
Claybourne Packaged Flower - Tahoe Dream - 3.5g - Smalls Sample
Leaf CA Packaged Flower - Mendo Breath - 3.5g
Claybourne Grinds - Rollex OG - Hybrid - 7g - (Case of 25)
Claybourne Packaged Flower - Empire Orange - 14g
Claybourne Power Pack - Samoa x Hybrid Kief - 3.5g - Flower - (Case of 20)
Claybourne Packaged Flower - Lemongrass - 14g - (Case of 8)
Claybourne Packaged Flower - Gorilla Cookies - 3.5g - Smalls Sample
Claybourne Power Pack - Apple Pie x Hybrid Kief - 3.5g - Flower - (Case of 20)
Claybourne Packaged Flower - Gelato Kush - 28g
C

In [116]:
s

{'Claybourne Packaged Flower - Orange Creamsicle - 14g - (Case of 8)',
 'Claybourne Packaged Flower - Oakland Purple - 28g - (Case of 4)',
 'Claybourne Grinds - OG Kush - Hybrid - 7g - Sample',
 'Claybourne Packaged Flower - Sundae Driver - 28g - (Case of 4)',
 'Claybourne Packaged Flower - Frosted Cherry Cookies - 28g - (Case of 4)',
 'Claybourne Packaged Flower - Strawberry Bang - 3.5g - Sample',
 'Claybourne Power Pack - WiFi x Hybrid Kief - 3.5g - Flower - (Case of 20)',
 'Claybourne Packaged Flower - Tahoe Dream - 3.5g - Smalls Sample',
 'Leaf CA Packaged Flower - Mendo Breath - 3.5g',
 'Claybourne Grinds - Rollex OG - Hybrid - 7g - (Case of 25)',
 'Claybourne Packaged Flower - Empire Orange - 14g',
 'Claybourne Power Pack - Samoa x Hybrid Kief - 3.5g - Flower - (Case of 20)',
 'Claybourne Packaged Flower - Lemongrass - 14g - (Case of 8)',
 'Claybourne Packaged Flower - Gorilla Cookies - 3.5g - Smalls Sample',
 'Claybourne Power Pack - Apple Pie x Hybrid Kief - 3.5g - Flower - (Ca

In [ ]:
CI Farms Bulk Flower
CI Farms Bulk Larf
Claybourne Bulk Flower
Claybourne Bulk Shake
Claybourne Bulk Smalls
Fleetwood Bulk Flower
Fleetwood Bulk Shake
Fleetwood Bulk Smalls

In [115]:
inv_name_set

{'0035 Smalls',
 '0035 flower',
 '0048 Smalls',
 '0048 flower',
 '0069 flower',
 '0102 flower',
 '0135 Flower',
 'CI Farms Bulk Flower - 21C',
 'CI Farms Bulk Flower - 3x Sour',
 'CI Farms Bulk Flower - Agent Orange',
 'CI Farms Bulk Flower - Alien OG',
 'CI Farms Bulk Flower - Animal Cookies',
 'CI Farms Bulk Flower - Animal Cookies x Kush Mints #11',
 'CI Farms Bulk Flower - Animal Tree',
 'CI Farms Bulk Flower - Apple Fritter x Sunset Sherbert',
 'CI Farms Bulk Flower - Appletartz',
 'CI Farms Bulk Flower - Banana Cream Cake x Jealousy',
 'CI Farms Bulk Flower - Blue Dream',
 'CI Farms Bulk Flower - Cali Sour',
 'CI Farms Bulk Flower - Caribbean Cookies',
 'CI Farms Bulk Flower - Chem Dawg',
 'CI Farms Bulk Flower - Chemistry',
 'CI Farms Bulk Flower - Cherry Cake',
 'CI Farms Bulk Flower - Cherry Cheesecake',
 'CI Farms Bulk Flower - Divine Storm',
 'CI Farms Bulk Flower - DoSaDo Sherbert',
 'CI Farms Bulk Flower - Gelato 41',
 'CI Farms Bulk Flower - Gelato 41 x Animal Mints',
 'C

In [97]:
sales_name_set = set(df_out_distro[df_out_distro['product_category_name'] == 'Flower']['product_name'].unique())

In [125]:
sales_name_set

{'Claybourne Packaged Flower - Orange Creamsicle - 14g - (Case of 8)',
 'Claybourne Packaged Flower - Oakland Purple - 28g - (Case of 4)',
 'Claybourne Bulk Flower - Afghani Bullrider',
 'Claybourne Bulk Smalls - Don Shula',
 'Claybourne Grinds - OG Kush - Hybrid - 7g - Sample',
 'Claybourne Packaged Flower - Sundae Driver - 28g - (Case of 4)',
 'Claybourne Packaged Flower - Frosted Cherry Cookies - 28g - (Case of 4)',
 'Claybourne Packaged Flower - Strawberry Bang - 3.5g - Sample',
 'Claybourne Power Pack - WiFi x Hybrid Kief - 3.5g - Flower - (Case of 20)',
 'Claybourne Packaged Flower - Tahoe Dream - 3.5g - Smalls Sample',
 'Leaf CA Packaged Flower - Mendo Breath - 3.5g',
 'Claybourne Bulk Smalls - Black Triangle OG',
 'CI Farms Bulk Flower - G41',
 'Claybourne Grinds - Rollex OG - Hybrid - 7g - (Case of 25)',
 'Claybourne Packaged Flower - Empire Orange - 14g',
 'Claybourne Power Pack - Samoa x Hybrid Kief - 3.5g - Flower - (Case of 20)',
 'Claybourne Packaged Flower - Lemongrass -

In [134]:
df_out_distro[(df_out_distro['product_category_name'] == 'Flower')&\
              (df_out_distro['per_unit_incoming'].notnull())&\
              (df_out_distro['per_unit_incoming'] > 0.1)]\
        [['create_year','per_unit_incoming']].groupby('create_year').mean().reset_index()


,create_year,per_unit_incoming
0,2020,3.035307


In [133]:
df_out_distro[(df_out_distro['product_category_name'] == 'Flower')&\
              (df_out_distro['per_unit_incoming'].notnull())&\
              (df_out_distro['per_unit_incoming'] > 0.1)]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name,create_year,per_unit_incoming
88838,OUTGOING_TO_PAYOR,C11-0000032-LIC,0001409231,2020-12-31,2021-01-05 01:35:38+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000482-LIC,"THE LEAF AT 73740, LLC",Wholesale Manifest,Wholesale,10477634,1A406030000326B000044573,transfer,1A406030000326B000044480,"DRB-N02.6, DRB-N03.7, DRB-N05.7",Accepted,False,False,Flower,Claybourne Packaged Flower - Durban Poison - 3...,passed,1.0,3.5,Grams,3.5,Grams,1.0,NaN,None,2020,0.285714
88842,OUTGOING_TO_PAYOR,C11-0000032-LIC,0001408757,2020-12-31,2021-01-05 00:14:34+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000242-LIC,"TORREY HOLISTICS, INC.",Wholesale Manifest,Wholesale,10475151,1A406030000326B000044512,transfer,1A406030000326B000042826,"H176 - FF, Kush Mints 10/6/20, P12-FF",Accepted,False,False,Flower,Claybourne Grinds - Wedding Crasher - Indica -...,passed,1.0,7.0,Grams,7.0,Grams,1.0,NaN,None,2020,0.142857
88844,OUTGOING_TO_PAYOR,C11-0000032-LIC,0001409113,2020-12-31,2021-01-04 22:24:24+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C12-0000119-LIC,Palm Desert Melanie Place Industrial LLC,Wholesale Manifest,Wholesale,10476482,1A406030000326B000044554,transfer,1A406030000326B000044477,"DRB-N02.6, DRB-N03.7, DRB-N05.7",Accepted,False,False,Flower,Claybourne Packaged Flower - Durban Poison - 3...,passed,3.0,10.5,Grams,10.5,Grams,3.0,NaN,None,2020,0.285714
88846,OUTGOING_TO_PAYOR,C11-0000032-LIC,0001408757,2020-12-31,2021-01-05 00:14:34+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000242-LIC,"TORREY HOLISTICS, INC.",Wholesale Manifest,Wholesale,10475150,1A406030000326B000044511,transfer,1A406030000326B000033009,Og Kush,Returned,False,False,Flower,Claybourne Grinds - OG Kush - Hybrid - 7g - (C...,passed,1250.0,700.0,Grams,NaN,None,NaN,NaN,None,2020,1.785714
88848,OUTGOING_TO_PAYOR,C11-0000032-LIC,0001408757,2020-12-31,2021-01-05 00:14:34+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000242-LIC,"TORREY HOLISTICS, INC.",Wholesale Manifest,Wholesale,10475154,1A406030000326B000044515,transfer,1A406030000326B000042825,"H176 - FF, Kush Mints 10/6/20, P12-FF",Accepted,False,False,Flower,Claybourne Grinds - Wedding Crasher - Indica -...,passed,937.5,525.0,Grams,525.0,Grams,937.5,NaN,None,2020,1.785714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123767,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000227096,2020-01-02,2020-01-04 00:31:56+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000121-LIC,"ECSD MANAGEMENT, LLC.",Wholesale Manifest,Wholesale,1793959,1A406030000326B000004269,transfer,1A406030000326B000002177,,Accepted,False,False,Flower,Claybourne Packaged Flower - GMO - 3.5g - Smal...,passed,1.0,3.5,Grams,3.5,Grams,1.0,NaN,None,2020,0.285714
123769,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000226079,2020-01-02,2020-01-02 23:52:20+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000298-LIC,"GSG PL, INC.",Wholesale Manifest,Wholesale,1785499,1A406030000326B000004239,transfer,1A406030000326B000002181,,Accepted,False,False,Flower,Claybourne Packaged Flower - Earthshaker OG - ...,passed,1.0,3.5,Grams,3.5,Grams,1.0,NaN,None,2020,0.285714
123771,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000227096,2020-01-02,2020-01-04 00:31:56+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C10-0000121-LIC,"ECSD MANAGEMENT, LLC.",Wholesale Manifest,Wholesale,1794637,1A406030000326B000004288,transfer,1A406030000326B000000016,

In [104]:
overlap_name_list= list(inv_name_set.intersection(sales_name_set))

In [90]:
df_inventory[df_inventory['product_name'] == 'Fleetwood Bulk Smalls - Pipe Dream']

,license_number,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date
756,C11-0000032-LIC,27227548,1A406030000326B000135978,active,2022-07-21,2022-07-21 23:00:14+00:00,Product,Fleetwood Bulk Smalls - Pipe Dream,Flower,258.0,Grams,4880403,Buds,20052522PD,20052522PD,PIDRG6B3052522,False,False,False,None,None


In [110]:
df_out_distro[df_out_distro['product_name'] == 'Claybourne Bulk Flower - Eagle MAC']

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name,create_year,per_unit_incoming
0,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120490,2022-10-01,2022-10-02 15:07:47+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000072-LIC,2 RIVER LABS,Transfer,Standard,29369135,1A406030000326B000146429,transfer,1A406030000326B000146192,13 9/16/2022,Accepted,True,False,Flower,Claybourne Bulk Flower - Eagle MAC,failed,NaN,20.00,Grams,20.00,Grams,NaN,NaN,None,2022,NaN
19012,OUTGOING_TO_PAYOR,C11-0000032-LIC,0003431516,2022-05-02,2022-05-04 00:01:36+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000013-LIC,SC LABORATORIES CALIFORNIA LLC,Transfer,Standard,24761244,1A406030000326B000124724,transfer,1A406030000326B000124478,Mac1 4/1/2022,Accepted,True,False,Flower,Claybourne Bulk Flower - Eagle MAC,failed,NaN,14.48,Grams,14.48,Grams,NaN,NaN,None,2022,NaN


In [108]:
df_out_distro[df_out_distro['product_name'].isin(overlap_name_list)]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name,create_year,per_unit_incoming
0,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120490,2022-10-01,2022-10-02 15:07:47+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000072-LIC,2 RIVER LABS,Transfer,Standard,29369135,1A406030000326B000146429,transfer,1A406030000326B000146192,13 9/16/2022,Accepted,True,False,Flower,Claybourne Bulk Flower - Eagle MAC,failed,NaN,20.0,Grams,20.0,Grams,NaN,NaN,None,2022,NaN
14,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120490,2022-10-01,2022-10-02 15:07:47+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000072-LIC,2 RIVER LABS,Transfer,Standard,29368589,1A406030000326B000146428,transfer,1A406030000326B000145821,12 9/16/2022,Accepted,True,False,Flower,Claybourne Bulk Flower - Apples and Bananas,failed,NaN,26.0,Grams,26.0,Grams,NaN,NaN,None,2022,NaN
16,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004120490,2022-10-01,2022-10-02 15:07:47+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000072-LIC,2 RIVER LABS,Transfer,Standard,29369125,1A406030000326B000146427,transfer,1A406030000326B000145820,7 9/16/2022,Accepted,True,False,Flower,Claybourne Bulk Flower - Apple Fritter x Sunse...,failed,NaN,22.0,Grams,22.0,Grams,NaN,NaN,None,2022,NaN
93,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004117266,2022-09-30,2022-09-30 16:34:57+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,CDPH-10004822,"CI Infusion, LLC",Transfer,Standard,29343878,1A406030000326B000146183,transfer,1A406030000326B000140605,Kush Mints 6/15/2022,Accepted,False,False,Flower,Claybourne Bulk Shake - Kush Mints,passed,NaN,3.0,Grams,3.0,Grams,NaN,NaN,None,2022,NaN
184,OUTGOING_TO_PAYOR,C11-0000032-LIC,0004117266,2022-09-30,2022-09-30 16:34:57+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,CDPH-10004822,"CI Infusion, LLC",Transfer,Standard,29343759,1A406030000326B000146185,transfer,1A406030000326B000137873,DRB-N04.14-221804,Accepted,False,False,Flower,Claybourne Bulk Shake - Durban Poison,passed,NaN,3.0,Grams,3.0,Grams,NaN,NaN,None,2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118272,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000372490,2020-03-06,2020-03-10 15:08:41+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000068-LIC,CALIFORNIA CANNABIS TESTING LABS,Transfer,Standard,3026287,1A406030000326B000009718,transfer,1A406030000326B000009665,Z3-I(CHARLES)-GMO,Accepted,True,False,Flower,Claybourne Bulk Smalls - GMO,failed,NaN,10.0,Grams,10.0,Grams,NaN,NaN,None,2020,NaN
119981,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000328183,2020-02-17,2020-02-17 23:55:46+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000040-LIC,CSA LA LLC,Transfer,Standard,2673268,1A406030000326B000008026,transfer,1A406030000326B000008007,Black Triangle - S012720 - Room 4 Harvest,Accepted,True,False,Flower,Claybourne Bulk Flower - Black Triangle OG,failed,NaN,44.4,Grams,44.4,Grams,NaN,NaN,None,2020,NaN
120092,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000323816,2020-02-14,2020-02-14 04:38:11+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000040-LIC,CSA LA LLC,Transfer,Standard,2634576,1A406030000326B000007795,transfer,1A406030000326B000007756,PCG Wedding Cake 01/15/20,Accepted,True,False,Flower,Claybourne Bulk Smalls - Wedding Cake,failed,NaN,19.7,Grams,19.7,Grams,NaN,NaN,None,2020,NaN
121188,OUTGOING_TO_PAYOR,C11-0000032-LIC,0000299846,2020-02-04,2020-02-05 13:00:14+00:00,C11-0000032-LIC,CI DISTRIBUTION LLC,C8-0000040-LIC,CSA LA LLC,Transfer,Standard,2447585,1A4

In [ ]:
#df = df_out_distro[df_out_distro['recipient_facility_license_number'].isin(retail_license_numbers)]

In [ ]:
inv_val_distro_to_retail['value'].values / inv_val_distro_to_retail['coverage'].values

In [ ]:
missing = inv_all_distro_to_retail[inv_all_distro_to_retail['per_unit_product'].isna()]

In [ ]:
missing[['product_name','unit_of_measure','quantity']].groupby(['product_name','unit_of_measure']).sum().sort_values(by = 'quantity',ascending = False)




In [ ]:
missing_prod_name = list(missing['product_name'].unique())

In [ ]:
len(missing_prod_name)

In [ ]:
missing[missing['product_name'] == 'GRIZZLY PEAK - Indica Bone - 0.5g - 7PK']

In [ ]:
# search for these products from GF's retail sales
# when they come in? new product?
#retailers should be selling them, do they appear in transactions?


In [ ]:
#df_sales_retail[df_sales_retail['tx_product_name'] == 'Firesale Panama Punch Sativa 5-Shot Prerolls Pack 3.5g	']

In [ ]:
df_in_all[df_in_all['product_name'].isin(missing_prod_name)][['package_lab_results_status','product_name']].\
groupby('package_lab_results_status').count()

In [ ]:
df_in_all[df_in_all['product_name'].isin(['Paletas Dosi Punch Indica Infused Moon Walkers Prerolls 5pk 2g',
       'Paletas - Biscotti X Sherbert X Jealousy - Infused Blunt - 2.1g',
       'Paletas Pink Panties Indica Flower 3.5g',
       'Paletas Maui Citrus Punch Sativa Infused Moonrock Blunt 1.5g',
       'Paletas Guava Sativa Infused Moon Walkers Prerolls 5pk 2g',
       'Paletas Jelly Donut Hybrid Infused Moonrock Blunt 1.5g',
       'Paletas Blackberry Kush Indica Infused Moon Walkers Prerolls 5pk 2g',
       'Paletas Watermelon OG Indica Infused Moonrock Blunt 1.5g',
       'Paletas Lime OG Hybrid Infused Moon Walkers Prerolls 5pk 2g',
       'Paletas Dosi Punch Indica Infused Moonrock Blunt 1.5g',
       'Paletas White Runtz Hybrid Infused Moonrock Blunt 1.5g',
       'Paletas - Horchata - Flower(3.5g)',
       'Paletas Haze Berry Sativa Infused Moon Walkers Prerolls 5pk 2g',
       'Paletas - Rainbow Runtz - Infused Blunt - 2.1g',
       'Paletas Grape Stomper Hybrid Infused Moon Walkers Prerolls 5pk 2g',
       'Paletas - Pink Cookies - Infused Blunt - 2.1g',
       'Paletas - Monster Cookies - Infused Blunt - 2.1g',
       'Paletas - Gelato 41 - Infused Blunt - 2.1g',
       'Paletas - Kush Mints - Infused Blunt - 2.1g',
       'Paletas - Orange Creamsicle - Infused Blunt - 2.1g',
       'Paletas - Peach Ringz - Infused Blunt - 2.1g',
       'Paletas - White Runtz - Infused Blunt - 2.1g',
       "Paletas - Mother's Milk - Infused Blunt - 2.1g",
       'Paletas - Tropaya - Infused Blunt - 2.1g',
       'Paletas - Cherry Pie - Infused Blunt - 2.1g',
       'LITLZ Biscotti 3.5g', 'LITLZ - Guava X Biscotti 3.5g',
       'Fire Sale - GSC 0.7g', 'Fire Sale - Lemon Meringue 0.7g',
       'Fire Sale 5 Shot - Indica - Dark Star 3.5g',
       'Fire Sale 5 Shot - Sativa - Lemon Dosi X Wookie 3.5g',
       'Fire Sale - Peanut Butter Breath 0.7g', 'LITLZ - Gelonade 3.5g',
       'Fire Sale - MAC 0.7g', 'LITLZ Gushers 3.5g',
       'Fire Sale - Dolato 0.7g', 'Fire Sale - Banana Cookies 0.7g',
       'Fire Sale - Acapulco Gold 0.7g', 'Fire Sale - Purple AK 0.7g',
       'Fire Sale 5 Shot - Indica - Eden Cake 3.5g',
       'Fire Sale - Citrus Punch 0.7g', 'Fire Sale - Tangie Cream 0.7g',
       'LITLZ Sherb Cake 3.5g', 'LITLZ Guava Kush 3.5g',
       "LITLZ Lamb's Bread 3.5g",
       'Old Pal | Clementine Sativa | 0.5g Cartridge',
       'Old Pal | Purple Punch Indica | 0.5g Cartridge'])]['shipper_facility_name'].unique()

In [ ]:
df_in_all[df_in_all['product_name'] == 'Firesale MAC Hybrid Preroll 0.7g' ]

In [ ]:
df_in_all[df_in_all['product_name'] == 'GRIZZLY PEAK - Indica Bone - 0.5g - 7PK' ]

In [ ]:
df_in_all[df_in_all['product_name'] == 'Firesale Mochi Runtz Hybrid 5-Shot Preroll Pack 3.5g' ]

In [ ]:
df_in_all[df_in_all['product_name'] == 'Firesale Panama Punch Sativa 5-Shot Prerolls Pack 3.5g' ]

## 2. distro inventory + cost based (cost to distro)

In [ ]:
inv_val_distro_cost,inv_all_distro_cost = client_surveillance_jupyter.calculate_inventory_valuation(df_in_distro,df_inventory,license_numbers,today)




In [ ]:
inv_val_distro_cost

In [ ]:
inv_val_distro_cost['value'].values / inv_val_distro_cost['coverage'].values

## 3. blended: distro inventory + all incoming data

In [ ]:
inv_val_all,inv_all = client_surveillance_jupyter.calculate_inventory_valuation(df_in_all,df_inventory,license_numbers,today)




In [ ]:
inv_val_all

In [ ]:
inv_val_all['value'].values / inv_val_all['coverage'].values